<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/text_analysis_coref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ru_core_news_sm
!python -m spacy download ru_core_news_md

     |████████████████████████████████| 1.6MB 4.6MB/s 
Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (56.0.0)
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 12.8 MB 94 kB/s 
     |████████████████████████████████| 1.1 MB 70.7 MB/s 
     |████████████████████████████████| 9.1 MB 55.7 MB/s 
     |████████████████████████████████| 456 kB 62.3 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 113 kB 58.4 MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=7604bd86f511534e3da5ade30439cf5a3ece6910d4b64b800635c76095908ab6
  Stored in directory: /root/.cache/pip/wheels/83/a6/12/bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open
  Attemp

In [2]:
pip install pymorphy2[fast]

     |████████████████████████████████| 371 kB 4.3 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=869302 sha256=6f8eed8d83f3d9fe6ee1e0136943d4c3f6210ffdaf96d0d0502ceb55e2227350
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [3]:
pip install Gensim

In [4]:
import spacy
import gensim
from gensim.models import Word2Vec 
from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd

In [5]:
nlp = spacy.load('ru_core_news_md')

In [6]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_ru.txt

--2021-05-02 18:41:16--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_ru.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3638613 (3.5M) [text/plain]
Saving to: ‘neuroleptic_ru.txt’

neuroleptic_ru.txt  100%[===================>]   3.47M  21.5MB/s    in 0.2s    

2021-05-02 18:41:17 (21.5 MB/s) - ‘neuroleptic_ru.txt’ saved [3638613/3638613]



In [7]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru.txt

--2021-05-02 18:41:17--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12963713 (12M) [text/plain]
Saving to: ‘rakpobedim_ru.txt’

rakpobedim_ru.txt   100%[===================>]  12.36M  47.2MB/s    in 0.3s    

2021-05-02 18:41:19 (47.2 MB/s) - ‘rakpobedim_ru.txt’ saved [12963713/12963713]



In [8]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit.txt

--2021-05-02 18:41:19--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6298252 (6.0M) [text/plain]
Saving to: ‘hv-info_gepatit.txt’

hv-info_gepatit.txt 100%[===================>]   6.01M  24.8MB/s    in 0.2s    

2021-05-02 18:41:20 (24.8 MB/s) - ‘hv-info_gepatit.txt’ saved [6298252/6298252]



In [9]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-05-02 18:41:20--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... failed: Connection timed out.
Retrying.

--2021-05-02 18:41:53--  (try: 2)  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz’

news_upos_cbow_300_ 100%[===================>] 133.81M  11.3MB/s    in 14s     

2021-05-02 18:44:25 (9.76 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz’ saved [140310878/140310878]



In [10]:
handler = open('neuroleptic_ru.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()


In [11]:
handler = open('hv-info_gepatit.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [12]:
handler = open('rakpobedim_ru.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [13]:
len(my_txt)

13016908

In [14]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-05-02 18:44:26--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
Unable to establish SSL connection.


In [15]:
posts = str_list = my_txt.split('----')

In [16]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]

In [17]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('news_upos_cbow_300_2_2017.bin.gz', binary=True)

In [18]:
model = Word2Vec(size=300, min_count=1)
model.build_vocab(sentences_tokenized)
total_examples = model.corpus_count
model.build_vocab([list(w2v.vocab.keys())], update=True)
model.intersect_word2vec_format("news_upos_cbow_300_2_2017.bin.gz", binary=True)
model.train(sentences_tokenized, total_examples=total_examples, epochs=model.iter)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


(8406266, 9683300)

In [19]:
model.most_similar('препарат')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('который', 0.7778945565223694),
 ('нельзя', 0.7754542827606201),
 ('зитига', 0.7150808572769165),
 ('дольше', 0.7128527760505676),
 ('его', 0.7123412489891052),
 ('резистентных', 0.7034340500831604),
 ('мощный', 0.70256108045578),
 ('сероквель', 0.6938760280609131),
 ('человек', 0.6905373334884644),
 ('счёт', 0.6897395253181458)]

In [20]:
# model['алимемазин']
model.wv.most_similar('алимемазин')

[('вскакиваете', 0.6121671199798584),
 ('пугают', 0.5965322852134705),
 ('снова', 0.5738897323608398),
 ('победил', 0.569255530834198),
 ('болезненно', 0.5656118392944336),
 ('взглянул', 0.5647401809692383),
 ('засыпала', 0.5540554523468018),
 ('нервничал', 0.5515797734260559),
 ('капелтница', 0.5427944660186768),
 ('будит', 0.539042592048645)]

In [21]:
cosine_similarity(np.atleast_2d(model.wv.get_vector('алимемазин')), \
                  np.atleast_2d(model.wv.get_vector('азалептин')))[0][0]

-0.0069493935

In [22]:
cosine_similarity(np.atleast_2d(model.wv.get_vector('алимемазин')), \
                  np.atleast_2d(model.wv.get_vector('стул')))[0][0]

0.16703528

In [23]:
# Модель есть, теперь надо набрать корпус из трех форумов примерно поровну, и чтобы не помереть при разметке.

СОВСЕМ НЕМНОЖКО, для начала!!


In [84]:
short_txt = " "
handler = open('neuroleptic_ru.txt', 'r', encoding='utf-8')
short_txt = handler.read()[:5000]
handler.close()

handler = open('rakpobedim_ru.txt', 'r', encoding='utf-8')
short_txt += handler.read()[:5000]
handler.close()

handler = open('hv-info_gepatit.txt', 'r', encoding='utf-8')
short_txt += handler.read()[:5000]
handler.close()

In [85]:
len(short_txt)

15000

In [86]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [189]:
short_posts = str_list = short_txt.split('----')
short_posts[10]

'\nНа рынок снова вернулся алимемазин (раньше терален назывался). Хотелось бы узнать Ваше мнение. Кто принимал в своей практике, что хорошего, что плохого\n\n____________________________________________\nСообщение отредактировано.\n\nДобавлена ссылка на ролик в You Tube.\n\nСообщение отредактировал Гилев: 12 Апрель 2020 - 07:55\nДобавлена ссылка на ролик в You Tube\n'

In [132]:
len(short_posts)


48

In [177]:
a = 3 if 3 else 6
a

3

In [178]:
# Словарь предложение:номер поста
tokenizer = RegexpTokenizer(r'\w+')
sent_dict = {}
for num, post in enumerate(short_posts):
  for sent in nltk.sent_tokenize(post):
    sent = ' '.join(tokenizer.tokenize(sent))
    sent = sent.replace('_', '') 
    # sent = nltk.regexp_tokenize(sent, '[а-яА-яa-zA-Z.!? ]')
    if sent:
      sent_dict[sent] = num 

In [190]:
len(sent_dict)

148

In [191]:
# только для удобства, если не будет влезать в память - использую ключи словаря
sent_list = list(sent_dict.keys())
sent_list[:9]

['На рынок снова вернулся алимемазин раньше терален назывался',
 'Хотелось бы узнать Ваше мнение',
 'Кто принимал в своей практике что хорошего что плохого  Сообщение отредактировано',
 'Добавлена ссылка на ролик в You Tube',
 'Сообщение отредактировал Гилев 12 Апрель 2020 07 55 Добавлена ссылка на ролик в You Tube',
 'Орто сказал а 27 11 2008 19 59 На рынок снова вернулся алимемазин раньше терален назывался',
 'Кто принимал в своей практике что хорошего что плохого Это тот редкий случай когда антипсихотик оставил самые приятные впечатления к сожалению про моно терапию рассказать не могу т к принимал его совместно с антидепрессантами Орто а где его можно приобрести не подскажете',
 'В Волгограде есть почти во всех аптеках стоит 80 руб',
 'Мы его применяемм в основном как вегетостабилизатор']

In [192]:
sent_dict[sent_list[0]]

10

In [193]:

# sentences_tokenized = [w.lower() for w in short_posts]
# sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]
# sentences_tokenized[0]

In [194]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [195]:
sample = 'их'
print('Одуш', morph.parse(sample)[0].tag.animacy)
print('Падеж', morph.parse(sample)[0].tag.case )
print('Пол', morph.parse(sample)[0].tag.gender)
print('Число', morph.parse(sample)[0].tag.number)
print('Лицо', morph.parse(sample)[0].tag.person)
print('Часть речи', morph.parse(sample)[0].tag.POS)
print('Время', morph.parse(sample)[0].tag.tense)
print('Залог', morph.parse(sample)[0].tag.voice)


Одуш None
Падеж accs
Пол None
Число plur
Лицо 3per
Часть речи NPRO
Время None
Залог None


In [196]:
sent_list[2]

'Кто принимал в своей практике что хорошего что плохого  Сообщение отредактировано'

In [197]:
doc = nlp(sent_list[2])

In [198]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_, 
            [child for child in token.children], morph.parse(token.text)[0].tag)


Кто nsubj принимал VERB [] NPRO,masc sing,nomn
принимал ROOT принимал VERB [Кто, практике, что, отредактировано] VERB,impf,tran masc,sing,past,indc
в case практике NOUN [] PREP
своей det практике NOUN [] ADJF,Apro,Anph femn,sing,gent
практике obl принимал VERB [в, своей] NOUN,inan,femn sing,loct
что obj принимал VERB [хорошего, что, Сообщение] CONJ
хорошего amod что SCONJ [] ADJF,Qual neut,sing,gent
что fixed что SCONJ [плохого,  ] CONJ
плохого amod что PRON [] ADJF,Qual neut,sing,gent
  punct что PRON [] UNKN
Сообщение case что SCONJ [] NOUN,inan,neut sing,nomn
отредактировано punct принимал VERB [] PRTS,perf,past,pssv neut,sing


In [108]:
def find_args(my_text, my_spacy):
  sents = text.split('.')
  for sent in sents:
    doc = my_spacy(u'Кто принимал его в своей практике')

In [201]:
token_list = [i for i in doc]

In [202]:
for token in doc:
  if token.dep_ == 'obj':
    print(token.text, token.dep_, token.head.text, token.head.pos_,
        [child for child in token.children])

что obj принимал VERB [хорошего, что, Сообщение]


***Разметка для кореференции***